<a href="https://colab.research.google.com/github/bioscienceresearch/Discovery_of_Merbecovirus_clone/blob/main/November_2019_Wuhan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Wuhan maps showing Connor Reed's work and apartment location

Maps accompany "Documentation of a laboratory-confirmed COVID-19 case with onset date in November 2019 and no known exposure to the Huanan Seafood Market"

*Kory Swain, Gilles Demaneuf, Steven E Massey, Adrian Jones, António Duarte, Steven Quay*

Imports

In [ ]:
!pip install --quiet pandas bokeh geopandas
!pip install --quiet python-dotenv

In [55]:
import pandas as pd
import geopandas as gpd
from bokeh.io import output_notebook
import os
import dotenv
import numpy as np

from bokeh.plotting import gmap
from bokeh.models import GMapOptions
from bokeh.models import ColumnDataSource

In [143]:
from bokeh.plotting import figure, show, output_notebook
from bokeh.tile_providers import get_provider, Vendors
from bokeh.models import GeoJSONDataSource
from bokeh.io import output_file, show
from bokeh.models.annotations import Label, LabelSet

Ingest

In [33]:
dotenv.load_dotenv('./env')

True

In [34]:
api_key = os.environ['GOOGLE_API_KEY']

In [92]:
output_notebook()
bokeh_width, bokeh_height = 1200,800

Ingest

In [78]:
df_poi = pd.read_csv('POI_WKT.txt')
df_poi.head()

,WKT,name
0,POINT (114.257296 30.619621 0.0),Huanan Seafood Market
1,POINT (114.373326 30.60695 0.0),Wufeng Jiayuan Apartment
2,POINT (114.373602 30.606126 0.0),Zhongbai Supermarket
3,POINT (114.379572 30.606883 0.0),Imagine English
4,POINT (114.347151 30.555579 0.0),Zhongnan hospital


In [79]:
s = gpd.GeoSeries.from_wkt(df_poi.WKT)
gdf = gpd.GeoDataFrame(data=df_poi, geometry=s)

In [80]:
gdf = gdf.set_crs('epsg:4326', allow_override=True)

In [81]:
gdf_webmerc = gdf['geometry'].to_crs(epsg=3857)
#gdf_webmerc.plot()

In [83]:
gdf_webmerc.head()

0    POINT Z (12719064.010 3583447.595 0.000)
1    POINT Z (12731980.411 3581808.634 0.000)
2    POINT Z (12732011.135 3581702.059 0.000)
3    POINT Z (12732675.712 3581799.968 0.000)
4    POINT Z (12729066.623 3575166.123 0.000)
Name: geometry, dtype: geometry

In [95]:
gdf['lon'] = gdf.geometry.apply(lambda p: p.x)
gdf['lat'] = gdf.geometry.apply(lambda p: p.y)

In [180]:
gdf.head()

,WKT,name,geometry,lon,lat
0,POINT (114.257296 30.619621 0.0),Huanan Seafood Market,POINT Z (114.25730 30.61962 0.00000),114.257296,30.619621
1,POINT (114.373326 30.60695 0.0),Wufeng Jiayuan Apartment,POINT Z (114.37333 30.60695 0.00000),114.373326,30.606950
2,POINT (114.373602 30.606126 0.0),Zhongbai Supermarket,POINT Z (114.37360 30.60613 0.00000),114.373602,30.606126
3,POINT (114.379572 30.606883 0.0),Imagine English,POINT Z (114.37957 30.60688 0.00000),114.379572,30.606883
4,POINT (114.347151 30.555579 0.0),Zhongnan hospital,POINT Z (114.34715 30.55558 0.00000),114.347151,30.555579


In [100]:
gdf_df=gdf.copy().drop(['WKT', 'geometry'], axis=1)

In [175]:
gdf_BSL=gdf_df[gdf_df['name'].isin(['CDC before Dec 2-2019', 'Wuhan University - ABSL3', 'WIV', 'CDC after Dec 2-2019'])]
gdf_BSL.loc[gdf_BSL["name"]=="WIV", "name"] = "WIV - BSL3"
gdf_markets=gdf_df[gdf_df['name'].isin(['Huanan Seafood Market'])]
gdf_hospital=gdf_df[gdf_df['name'].isin(['Zhongnan hospital'])]
gdf_apartment=gdf_df[gdf_df['name'].isin(["Connor Reed"])]
gdf_imagine=gdf_df[gdf_df['name'].isin(['Imagine English'])]
gdf_interest=gdf_df[~gdf_df['name'].isin(['CDC before Dec 2-2019', 'Wuhan University - ABSL3', 'WIV',\
                                          'CDC after Dec 2-2019', 'Huanan Seafood Market', 'Zhongnan hospital', 'Imagine English'])]


In [119]:
lat_max, lat_min=gdf_df['lat'].max(), gdf_df['lat'].min()
lon_max, lon_min=gdf_df['lon'].max(), gdf_df['lon'].min()
lat_mid=lat_min+(lat_max-lat_min)/2
lon_mid=lon_min+(lon_max-lon_min)/2

In [136]:
['lat'][0]

'lat'

Maps

In [153]:
def plot(lat, lng, zoom=12, map_type='roadmap'):
    gmap_options = GMapOptions(lat=lat, lng=lng,
                               map_type=map_type, zoom=zoom)
    p = gmap(api_key, gmap_options, title='Wuhan',
             width=bokeh_width, height=bokeh_height)
    source = ColumnDataSource(gdf_interest)
    center = p.circle('lon', 'lat', size=4, alpha=0.9,
                      color='yellow', source=source)

    market = ColumnDataSource(gdf_markets)
    center = p.triangle('lon', 'lat', size=8, alpha=0.9,
                      color='red', source=market)
    label_mkt = LabelSet(x='lon', y='lat', text='name', level='overlay', text_align='center',
              x_offset=0, y_offset=0.05, text_color='white', source=market)
    p.add_layout(label_mkt)

    bsl_lab = ColumnDataSource(gdf_BSL)
    center = p.diamond('lon', 'lat', size=8, alpha=0.9,
                      color='orange', source=bsl_lab)
    labels_bsl = LabelSet(x='lon', y='lat', text='name', level='overlay', text_align='center',
              x_offset=0, y_offset=0.05, text_color='white', source=bsl_lab)
    p.add_layout(labels_bsl)

    hospital = ColumnDataSource(gdf_hospital)
    center = p.circle('lon', 'lat', size=8, alpha=0.9,
                      color='white', source=gdf_hospital)
    labels_hsp = LabelSet(x='lon', y='lat', text='name', level='overlay', text_align='center',
              x_offset=0, y_offset=0.05, text_color='white', source=hospital)
    p.add_layout(labels_hsp)

    imagine = ColumnDataSource(gdf_imagine)
    center = p.circle('lon', 'lat', size=8, alpha=0.9,
                      color='blue', source=imagine)
    labels_imagine = LabelSet(x='lon', y='lat', text='name', level='overlay', text_align='center',
              x_offset=0, y_offset=0.05, text_color='white', source=imagine)
    p.add_layout(labels_imagine)

    show(p)
    return p

p = plot(lat_mid, lon_mid, map_type='satellite')

Wuhan city map showing locations visited by Connor Reed in yellow

In [179]:
def plot(lat, lng, zoom=13, map_type='roadmap'):
    gmap_options = GMapOptions(lat=lat, lng=lng,
                               map_type=map_type, zoom=zoom)
    p = gmap(api_key, gmap_options, title='Wuhan',
             width=bokeh_width, height=bokeh_height)
    source = ColumnDataSource(gdf_interest)
    center = p.circle('lon', 'lat', size=4, alpha=0.9,
                      color='yellow', source=source)

    market = ColumnDataSource(gdf_markets)
    center = p.triangle('lon', 'lat', size=8, alpha=0.9,
                      color='red', source=market)
    label_mkt = LabelSet(x='lon', y='lat', text='name', level='overlay', text_align='center',
              x_offset=0, y_offset=0.05, text_color='white', source=market)
    p.add_layout(label_mkt)

    bsl_lab = ColumnDataSource(gdf_BSL)
    center = p.diamond('lon', 'lat', size=8, alpha=0.9,
                      color='orange', source=bsl_lab)
    labels_bsl = LabelSet(x='lon', y='lat', text='name', level='overlay', text_align='center',
              x_offset=0, y_offset=0.05, text_color='white', source=bsl_lab)
    p.add_layout(labels_bsl)

    hospital = ColumnDataSource(gdf_hospital)
    center = p.circle('lon', 'lat', size=8, alpha=0.9,
                      color='white', source=gdf_hospital)
    labels_hsp = LabelSet(x='lon', y='lat', text='name', level='overlay', text_align='center',
              x_offset=0, y_offset=0.05, text_color='white', source=hospital)
    p.add_layout(labels_hsp)

    imagine = ColumnDataSource(gdf_imagine)
    center = p.circle('lon', 'lat', size=8, alpha=0.9,
                      color='blue', source=imagine)
    labels_imagine = LabelSet(x='lon', y='lat', text='name', level='overlay', text_align='center',
              x_offset=0, y_offset=0.05, text_color='white', source=imagine)
    p.add_layout(labels_imagine)

    show(p)
    return p

p = plot(lat_mid, lon_mid, map_type='satellite')

As previous map, zoomed in

In [176]:
def plot(lat, lng, zoom=17, map_type='roadmap'):
    gmap_options = GMapOptions(lat=lat, lng=lng,
                               map_type=map_type, zoom=zoom)
    p = gmap(api_key, gmap_options, title='Wuhan',
             width=bokeh_width, height=bokeh_height)
    source = ColumnDataSource(gdf_interest)
    center = p.circle('lon', 'lat', size=4, alpha=0.9,
                      color='yellow', source=source)

    apartment = ColumnDataSource(gdf_apartment)
    center = p.triangle('lon', 'lat', size=4, alpha=0.9,
                      color='red', source=apartment)
    labels = LabelSet(x='lon', y='lat', text='name', level='overlay', text_align='center',
              x_offset=0, y_offset=0.05, text_color='white', source=apartment)
    p.add_layout(labels)

    imagine = ColumnDataSource(gdf_imagine)
    center = p.circle('lon', 'lat', size=8, alpha=0.9,
                      color='blue', source=imagine)
    labels_imagine = LabelSet(x='lon', y='lat', text='name', level='overlay', text_align='center',
              x_offset=0, y_offset=0.05, text_color='white', source=imagine)
    p.add_layout(labels_imagine)

    show(p)
    return p



In [177]:
p = plot(30.606, 114.375, map_type='satellite')

Local area around Connor's apartment and Imagine English